In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline

from statsmodels.formula.api import ols
import statsmodels.api as sm
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import LabelBinarizer

In [2]:
df = pd.read_csv('DataTable.csv')
df_question1=df

In [3]:
df.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15', 'distance_to_airport',
       'distance_to_seahawks_stadium', 'distance_to_seattle_center',
       'distance_to_kerry_park', 'distance_to_discovery_park',
       'minimum_distance_to_metro', 'population', 'land', 'population_density',
       'units', 'units_occupied', 'med_value', 'med_income', 'calc_pd',
       'walk_score'],
      dtype='object')

In [4]:
#drop columns that won't be useful or are repeated
df.drop(columns=['id', 'date', 'zipcode', 'units', 'population', 
                 'units_occupied', 'med_value', 'population_density', 'land'], inplace=True)

df.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,sqft_lot15,distance_to_airport,distance_to_seahawks_stadium,distance_to_seattle_center,distance_to_kerry_park,distance_to_discovery_park,minimum_distance_to_metro,med_income,calc_pd,walk_score
0,221900.0,3,1.00,1180,5650,1.0,0.0,0.0,3,7,...,5650,4.862539,6.771217,8.837087,9.483657,12.249694,1.302968,58707.0,4967.42268,39
1,205425.0,2,1.00,880,6780,1.0,0.0,0.0,4,6,...,6780,4.514162,7.598140,9.663983,10.310590,13.074902,2.089548,58707.0,4967.42268,62
2,445000.0,3,2.25,2100,8201,1.0,0.0,2.0,3,8,...,8712,5.076186,7.222682,9.278340,9.927283,12.720516,1.894527,58707.0,4967.42268,23
3,236000.0,3,1.00,1300,5898,1.0,0.0,0.0,3,7,...,7619,4.564977,7.170015,9.237631,9.883293,12.639671,1.643102,58707.0,4967.42268,29
4,170000.0,2,1.00,860,5265,1.0,0.0,0.0,3,6,...,8775,4.147126,6.839175,8.907012,9.545065,12.244964,1.264256,58707.0,4967.42268,80


In [5]:
df.describe()


,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,sqft_lot15,distance_to_airport,distance_to_seahawks_stadium,distance_to_seattle_center,distance_to_kerry_park,distance_to_discovery_park,minimum_distance_to_metro,med_income,calc_pd,walk_score
count,2.142000e+04,21420.000000,21420.000000,21420.000000,2.142000e+04,21420.000000,21420.000000,21420.000000,21420.000000,21420.000000,...,21420.000000,21420.000000,21420.000000,21420.000000,21420.000000,21420.000000,21420.000000,21420.000000,21420.000000,21420.000000
mean,5.407393e+05,3.373950,2.118429,2083.132633,1.512804e+04,1.495985,0.006816,0.233987,3.410784,7.662792,...,12775.718161,13.330562,11.408495,12.013397,12.284533,13.983176,6.970616,81151.809944,4294.850826,40.003361
std,3.679311e+05,0.925405,0.768720,918.808412,4.153080e+04,0.540081,0.082280,0.765437,0.650035,1.171971,...,27345.621867,5.912335,6.406267,7.098593,7.323247,8.042231,4.979949,24184.833310,2959.739674,28.561745
min,7.800000e+04,1.000000,0.500000,370.000000,5.200000e+02,1.000000,0.000000,0.000000,1.000000,3.000000,...,651.000000,0.794968,0.781762,0.328556,0.086093,0.156062,0.039381,40815.000000,41.211905,0.000000
25%,3.225000e+05,3.000000,1.750000,1430.000000,5.040000e+03,1.000000,0.000000,0.000000,3.000000,7.000000,...,5100.000000,8.405874,6.308190,6.177725,6.415543,7.475380,3.335343,63709.000000,2185.126582,14.000000
50%,4.500000e+05,3.000000,2.250000,1920.000000,7.614000e+03,1.500000,0.000000,0.000000,3.000000,7.000000,...,7620.000000,13.487255,10.390095,10.670040,10.919394,13.131834,5.974098,77688.000000,3975.248933,36.000000
75%,6.450000e+05,4.000000,2.500000,2550.000000,1.069050e+04,2.000000,0.000000,0.000000,4.000000,8.000000,...,10086.250000,17.535100,15.453381,16.681599,17.154461,19.658747,9.415636,95043.000000,6311.948867,65.000000
max,7.700000e+06,33.000000,8.000000,13540.000000,1.651359e+06,3.500000,1.000000,4.000000,5.000000,13.000000,...,871200.000000,49.875319,48.146469,48.787233,49.098762,51.018345,46.338136,183833.000000,15844.274809,99.000000


In [6]:
df.bedrooms.value_counts()

3     9731
4     6849
2     2736
5     1586
6      265
1      191
7       38
8       13
9        6
10       3
11       1
33       1
Name: bedrooms, dtype: int64

In [7]:
df.loc[df['bedrooms'] > 12]

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,sqft_lot15,distance_to_airport,distance_to_seahawks_stadium,distance_to_seattle_center,distance_to_kerry_park,distance_to_discovery_park,minimum_distance_to_metro,med_income,calc_pd,walk_score
5368,640000.0,33,1.75,1620,6000,1.0,0.0,0.0,5,7,...,4700,16.44744,6.397437,4.654191,4.24761,4.065119,2.91604,75763.0,9894.612069,65


In [8]:
df.loc[df['bedrooms'] > 12, 'bedrooms'] = 3

In [9]:
df.describe()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,sqft_lot15,distance_to_airport,distance_to_seahawks_stadium,distance_to_seattle_center,distance_to_kerry_park,distance_to_discovery_park,minimum_distance_to_metro,med_income,calc_pd,walk_score
count,2.142000e+04,21420.000000,21420.000000,21420.000000,2.142000e+04,21420.000000,21420.000000,21420.000000,21420.000000,21420.000000,...,21420.000000,21420.000000,21420.000000,21420.000000,21420.000000,21420.000000,21420.000000,21420.000000,21420.000000,21420.000000
mean,5.407393e+05,3.372549,2.118429,2083.132633,1.512804e+04,1.495985,0.006816,0.233987,3.410784,7.662792,...,12775.718161,13.330562,11.408495,12.013397,12.284533,13.983176,6.970616,81151.809944,4294.850826,40.003361
std,3.679311e+05,0.902995,0.768720,918.808412,4.153080e+04,0.540081,0.082280,0.765437,0.650035,1.171971,...,27345.621867,5.912335,6.406267,7.098593,7.323247,8.042231,4.979949,24184.833310,2959.739674,28.561745
min,7.800000e+04,1.000000,0.500000,370.000000,5.200000e+02,1.000000,0.000000,0.000000,1.000000,3.000000,...,651.000000,0.794968,0.781762,0.328556,0.086093,0.156062,0.039381,40815.000000,41.211905,0.000000
25%,3.225000e+05,3.000000,1.750000,1430.000000,5.040000e+03,1.000000,0.000000,0.000000,3.000000,7.000000,...,5100.000000,8.405874,6.308190,6.177725,6.415543,7.475380,3.335343,63709.000000,2185.126582,14.000000
50%,4.500000e+05,3.000000,2.250000,1920.000000,7.614000e+03,1.500000,0.000000,0.000000,3.000000,7.000000,...,7620.000000,13.487255,10.390095,10.670040,10.919394,13.131834,5.974098,77688.000000,3975.248933,36.000000
75%,6.450000e+05,4.000000,2.500000,2550.000000,1.069050e+04,2.000000,0.000000,0.000000,4.000000,8.000000,...,10086.250000,17.535100,15.453381,16.681599,17.154461,19.658747,9.415636,95043.000000,6311.948867,65.000000
max,7.700000e+06,11.000000,8.000000,13540.000000,1.651359e+06,3.500000,1.000000,4.000000,5.000000,13.000000,...,871200.000000,49.875319,48.146469,48.787233,49.098762,51.018345,46.338136,183833.000000,15844.274809,99.000000


In [10]:
#start dropping rows
df.groupby('floors')['price', 'sqft_living'].describe()

price                                                            \
          count          mean           std       min       25%       50%   
floors                                                                      
1.0     10552.0  4.424419e+05  2.644318e+05   78000.0  276000.0  390000.0   
1.5      1888.0  5.598977e+05  3.046153e+05   92000.0  355000.0  525000.0   
2.0      8203.0  6.490914e+05  4.343605e+05   90000.0  379000.0  542500.0   
2.5       161.0  1.061021e+06  8.588360e+05  255000.0  550000.0  799200.0   
3.0       609.0  5.832662e+05  3.391848e+05  205000.0  385000.0  491000.0   
3.5         7.0  9.102143e+05  1.054669e+06  435000.0  502000.0  525000.0   

                             sqft_living                                    \
              75%        max       count         mean          std     min   
floors                                                                       
1.0      530000.0  4670000.0     10552.0  1734.684325   703.299362   370.0   
1.5      688597.0  3650000.0      1888.0  1910.324153   700.119780   490.0   
2.0      772750.0  7060000.0      8203.0  2571.761673   955.139376   550.0   
2.5     1300000.0  7700000.0       161.0  3145.428571  1434.995822   990.0   
3.0      631500.0  3100000.0       609.0  1789.761905   961.892816   760.0   
3.5      553750.0  3300000.0         7.0  2440.000000  2331.022665  1310.0   

                                         
           25%     50%     75%      max  
floors                                   
1.0     1220.0  1640.0  2110.0   9640.0  
1.5     1400.0  1760.0  2290.0   5530.0  
2.0     1890.0  2440.0  3090.0  10040.0  
2.5     2170.0  2850.0  3760.0  12050.0  
3.0     1310.0  1500.0  1810.0  13540.0  
3.5     1420.0  1730.0  1745.0   7710.0

In [11]:
df = df.drop(df[df.floors > 3].index)

In [12]:
df.loc[df['floors'] == 1.5, 'floors'] = 2
df.loc[df['floors'] == 2.5, 'floors'] = 3

We'll use the `yr_built` column to calculate the age of the home and use age as a continuous variable.  We'll also create a new variable called `renovation_age` which is calculated in the same way.  If the house has not been renovated then it will have a renovation age equal to 0.

In [13]:
def calc_age(x):
    """Funtion calculates age of the house """
    if x == 0:
        age = 0
    else:
        age = 2020 - x
    return age
df['renovation_age'] = df['yr_renovated'].map(calc_age)
df['home_age'] = df['yr_built'].map(calc_age)
df.drop(columns=['yr_built', 'yr_renovated'], inplace=True)
df.head()


,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,distance_to_seahawks_stadium,distance_to_seattle_center,distance_to_kerry_park,distance_to_discovery_park,minimum_distance_to_metro,med_income,calc_pd,walk_score,renovation_age,home_age
0,221900.0,3,1.00,1180,5650,1.0,0.0,0.0,3,7,...,6.771217,8.837087,9.483657,12.249694,1.302968,58707.0,4967.42268,39,0.0,65
1,205425.0,2,1.00,880,6780,1.0,0.0,0.0,4,6,...,7.598140,9.663983,10.310590,13.074902,2.089548,58707.0,4967.42268,62,0.0,75
2,445000.0,3,2.25,2100,8201,1.0,0.0,2.0,3,8,...,7.222682,9.278340,9.927283,12.720516,1.894527,58707.0,4967.42268,23,0.0,53
3,236000.0,3,1.00,1300,5898,1.0,0.0,0.0,3,7,...,7.170015,9.237631,9.883293,12.639671,1.643102,58707.0,4967.42268,29,0.0,59
4,170000.0,2,1.00,860,5265,1.0,0.0,0.0,3,6,...,6.839175,8.907012,9.545065,12.244964,1.264256,58707.0,4967.42268,80,0.0,89


In [ ]:
#check the distributions of each of the columns
df.drop(columns=['lat', 'long']).hist(figsize=(18,16));
plt.tight_layout()

In [ ]:
# Check scatter plot of each column with continuos data
cont_cols = ['calc_pd', 'distance_to_airport', 'distance_to_discovery_park', 'distance_to_kerry_park',
             'distance_to_seahawks_stadium', 'distance_to_seattle_center', 'home_age', 'med_income',
             'minimum_distance_to_metro', 'renovation_age', 'sqft_above', 'sqft_basement', 'sqft_living',
             'sqft_living15', 'sqft_lot', 'sqft_lot15', 'walk_score']
plt.figure(figsize=(20,20))
for i in range(0, 20):
    plt.subplot(5, 4, i+1)
    if i > 16:
        break
    plt.scatter(df[cont_cols[i]], df['price'])
    plt.xlabel(cont_cols[i])
    plt.ylabel('Price')
plt.tight_layout()

In [ ]:
# Check scatter plot of each column with categorical data
cat_cols = ['bathrooms', 'bedrooms', 'condition', 'floors', 'grade', 'view', 'waterfront']
plt.figure(figsize=(20,20))
for i in range(0, 8):
    plt.subplot(4, 2, i+1)
    if i > 6:
        break
    plt.scatter(df[cat_cols[i]], df['price'])
    plt.xlabel(cat_cols[i])
    plt.ylabel('Price')
plt.tight_layout()

In [ ]:
df = df.loc[df.bedrooms <= 6]

In [ ]:
df = df.loc[df.bathrooms <= 5]

In [ ]:
plt.figure(figsize=(20,20))
sns.heatmap(df.drop(columns=['lat', 'long']).corr(), annot=True);

In [ ]:
# Remove columns we dont intend to use or have to high multicollinearity 
df.drop(columns=['condition', 'distance_to_discovery_park', 'distance_to_kerry_park', 
                'distance_to_seahawks_stadium', 'minimum_distance_to_metro', 'renovation_age',
                'sqft_living', 'sqft_living15', 'sqft_lot', 'sqft_lot15', 'view', 'waterfront',
                'grade', 'lat', 'long'], inplace=True)


In [ ]:
X = df.drop(columns=['price'])
vif = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
list(zip(X.columns, vif))

In [ ]:
#Calculate a new metric for Bathroom and Bedrooms
df['bed_bath'] = df['bedrooms'] / df['bathrooms']

In [ ]:
#plt.figure(figsize=(20,20))
#sns.heatmap(df.corr(), annot=True);

In [ ]:
df.drop(columns=['bedrooms', 'bathrooms'], inplace=True)

In [ ]:
len(df)

In [ ]:
#plt.figure(figsize=(20,20))
#sns.heatmap(df.corr(), annot=True);


In [ ]:
df.loc[df['sqft_basement']].describe()

In [ ]:
df['sqft_basement'] = df['sqft_basement'].astype(dtype='int64')

In [ ]:
#Categorize basement area into 4 categories 
df.loc[df['sqft_basement'] < 1, 'sqft_basement'] = 1
df.loc[((df['sqft_basement'] > 1) & (df['sqft_basement'] <=450)) , 'sqft_basement'] = 2
df.loc[((df['sqft_basement'] > 450) & (df['sqft_basement'] <= 970)), 'sqft_basement'] = 3
df.loc[df['sqft_basement'] > 970, 'sqft_basement'] = 4

In [ ]:
bsmt_cat = df['sqft_basement'].astype('category')
bsmt_dum_df = pd.get_dummies(bsmt_cat, prefix='bsmt_val', drop_first=True)
df = pd.concat([df, bsmt_dum_df], axis=1)

In [ ]:
#Categorize floors
floors_cat = df['floors'].astype('category')
floors_dum_df = pd.get_dummies(floors_cat, prefix='story', drop_first=True)
df = pd.concat([df, floors_dum_df], axis=1)

In [ ]:
df.columns

In [ ]:
df.rename(columns={'story_2.0': 'two_story', 'story_3.0': 'three_story'},inplace=True)
df.rename(columns={'bsmt_val_2': 'bsmt_med', 'bsmt_val_3':'bsmt_large', 'bsmt_val_4':'bsmt_xlarge'}, inplace=True)

In [ ]:
df.drop(columns=['floors', 'sqft_basement'], inplace=True)


In [ ]:
#plt.figure(figsize=(20,20))
#sns.heatmap(df.corr(), annot=True);

In [ ]:
# Check for multicollinearity using VIF
X = df.drop(columns=['price'])
vif = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
list(zip(X.columns, vif))

In [ ]:
outcome = 'price'
predictors = df.drop('price', axis=1)
pred_sum = '+'.join(predictors.columns)
formula = outcome + '~' + pred_sum

In [ ]:
model_vanilla1 = ols(formula=formula, data=df).fit()
model_vanilla1.summary()

In [ ]:
#look at distributions to check for outliers
fig, axes = plt.subplots(3, 1, figsize=(8, 10))
sns.distplot(df['price'], ax=axes[0])
sns.distplot(df['sqft_above'], ax=axes[1])
sns.distplot(df['bed_bath'], ax=axes[2])

plt.tight_layout()

In [ ]:
df['price'].describe()

In [ ]:
# Outlier metric using quantiles for 2 St dev.
upper_price = df['price'].quantile([0.975])
lower_price = df['price'].quantile([0.025])
print(upper_price)
print(lower_price)

In [ ]:
#eliminate price outliers
df = df.loc[(df.price < 1490000) & (df.price > 185000)]

In [ ]:
upper_price = df['price'].quantile([0.975])
lower_price = df['price'].quantile([0.025])
print(upper_price)
print(lower_price)

In [ ]:
df = df.loc[df.price < 1170000]

In [ ]:
upper_sqft = df['sqft_above'].quantile([0.997])
lower_sqft = df['sqft_above'].quantile([0.003])
print(upper_sqft)
print(lower_sqft)

In [ ]:
upper_bed_bath = df['bed_bath'].quantile([0.997])
lower_bed_bath = df['bed_bath'].quantile([0.003])
print(upper_bed_bath)
print(lower_bed_bath)

In [ ]:
df.loc[(df['bed_bath'] < 1) & (df['bed_bath'] > 0.5)].describe()

In [ ]:
df.loc[df.bed_bath == 4].describe()

In [ ]:
df.loc[df.bed_bath > 4].describe()

In [ ]:
df = df.loc[(df.bed_bath < 4) & (df.bed_bath > 1)]

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(8, 10))
sns.distplot(df['price'], ax=axes[0])
sns.distplot(df['sqft_above'], ax=axes[1])
sns.distplot(df['bed_bath'], ax=axes[2])

plt.tight_layout()

In [ ]:
df.loc[(df.sqft_above < 610) | (df.sqft_above > 4500)].describe()

In [ ]:
df = df.loc[(df.sqft_above > 610) & (df.sqft_above < 4500)]

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(8, 10))
sns.distplot(df['price'], ax=axes[0])
sns.distplot(df['sqft_above'], ax=axes[1])
sns.distplot(df['bed_bath'], ax=axes[2])

plt.tight_layout()

In [ ]:
len(df)

In [ ]:
df.describe()

In [ ]:
vanilla_model2 = ols(formula=formula, data=df).fit()
vanilla_model2.summary()

In [ ]:
plt.figure(figsize=(20,20))
sns.heatmap(df.corr(), annot=True);

In [ ]:
X = df.drop(columns=['price'])
vif = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
list(zip(X.columns, vif))

In [ ]:
df_drop1 = df.drop(columns=['distance_to_airport', 'calc_pd'])

In [ ]:
X = df_drop1.drop(columns=['price'])
vif = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
list(zip(X.columns, vif))

In [ ]:
df_drop2 = df_drop1.drop(columns=['walk_score'])

In [ ]:
X = df_drop2.drop(columns=['price'])
vif = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
list(zip(X.columns, vif))

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(df_drop2.corr(), annot=True);

In [ ]:
outcome = 'price'
predictors = df_drop2.drop('price', axis=1)
pred_sum = '+'.join(predictors.columns)
formula = outcome + '~' + pred_sum

vanilla_model3 = ols(formula=formula, data=df_drop1).fit()
vanilla_model3.summary()

In [ ]:
df_high_pval = df_drop2.drop(columns=['two_story'])
df_high_pval.head()

In [ ]:
#Log transform 
df_log = pd.DataFrame()
df_log['price'] = df_high_pval['price'].map(lambda x: np.log(x))
df_log['sqft_above'] = df_high_pval['sqft_above'].map(lambda x: np.log(x))
#df_log['sqft_basement'] = df_high_pval['sqft_basement'].map(lambda x: np.log(x+1))
df_log['dist_to_seattle_center'] = df_high_pval['distance_to_seattle_center'].map(lambda x: np.log(x))
df_log['med_income'] = df_high_pval['med_income'].map(lambda x: np.log(x))
df_log['home_age'] = df_high_pval['home_age'].map(lambda x: np.log(x))
df_log['bed_bath'] = df_high_pval['bed_bath'].map(lambda x: np.log(x))
#df_log['two_story'] = df_high_pval['two_story']
df_log['bsmt_med'] = df_high_pval['bsmt_med']
df_log['bsmt_large'] = df_high_pval['bsmt_large']
df_log['bsmt_xlarge'] = df_high_pval['bsmt_xlarge']
df_log['three_story'] = df_high_pval['three_story']

df_log.head()

In [ ]:
# Scale data 
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled = scaler.fit_transform(df_log)
scaled_df = pd.DataFrame(scaled)
column_dict = dict(zip(scaled_df.columns, df_log.columns))

scaled_df.rename(columns=column_dict, inplace=True)
scaled_df.head()

In [ ]:
outcome = 'price'
predictors_scaled = scaled_df.drop(['price'], axis=1)
pred_scaled_sum = '+'.join(predictors_scaled.columns)
formula_scaled = outcome + '~' + pred_scaled_sum

model4 = ols(formula = formula_scaled, data=scaled_df).fit()
model4.summary()

In [ ]:
import scipy.stats as stats
resid = model4.resid
fig = sm.graphics.qqplot(resid, dist=stats.norm, line='45', fit=True)

In [ ]:
X = scaled_df.drop(columns=['price'])
vif = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
list(zip(X.columns, vif))

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(scaled_df.corr(), annot=True);

In [ ]:
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.diagnostic import het_white

white_test = het_white(resid, model4.model.exog)
bp_test = het_breuschpagan(resid, model4.model.exog)

labels = ['LM Statistic', 'LM-Test p-value', 'F-Statistic', 'F-Test p-value']
print(dict(zip(labels, bp_test)))
print(dict(zip(labels, white_test)))

In [ ]:
for x in predictors_scaled.columns:
    fig = plt.figure(figsize=(10,8))
    fig = sm.graphics.plot_regress_exog(model4, x, fig=fig)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

linreg = LinearRegression()
X = scaled_df.drop(columns=['price'])
y  = scaled_df['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)
linreg.fit(X_train, y_train)
y_hat_train = linreg.predict(X_train)
y_hat_test = linreg.predict(X_test)

print(mean_squared_error(y_train, y_hat_train))
print(mean_squared_error(y_test, y_hat_test))


In [ ]:
from sklearn.model_selection import cross_val_score
cv_10_results = np.mean(cross_val_score(linreg, X, y, cv=10, scoring='neg_mean_squared_error'))
cv_20_results = np.mean(cross_val_score(linreg, X, y, cv=20, scoring='neg_mean_squared_error'))

In [ ]:
print(cv_10_results)
print(cv_20_results)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
print(mean_squared_error(y, linreg.predict(X)))
print(r2_score(y, linreg.predict(X)))

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score


# We'll fit 3 different polynomial regression models from degree 2 to degree 4
for index, degree in enumerate([2, 3, 4, 5]):
    
    # Instantiate PolynomialFeatures
    poly = PolynomialFeatures(degree)
    
    # Fit and transform X_train
    X_poly_train = poly.fit_transform(X_train)
    
    # Instantiate and fit a linear regression model to the polynomial transformed train features
    reg_poly = LinearRegression().fit(X_poly_train, y_train)
    
    # Transform the test data into polynomial features
    X_poly_test = poly.transform(X_test)
    
    # Get predicted values for transformed polynomial test data  
    y_pred = reg_poly.predict(X_poly_test)
    
    # Evaluate model performance on test data
    print("degree %d" % degree, r2_score(y_test, y_pred))
    print("degree %d" % degree, mean_squared_error(y_test, y_pred))
    
    # Transform the full data
    X_poly = poly.transform(X)
    
    # Now, we want to see what the model predicts for the entire data  
    y_poly = reg_poly.predict(X_poly)
    
    # Create plot of predicted values
    #plt.plot(X, y_poly, color = colors[index], linewidth=2, label='degree %d' % degree)
    #plt.legend(loc='lower left')


In [ ]:
poly2 = PolynomialFeatures(2)
X_degree2 = poly2.fit_transform(X)
reg_poly2 = LinearRegression().fit(X_degree2, y)

from sklearn.model_selection import cross_val_score

cv_5_results  = np.mean(cross_val_score(reg_poly2, X, y, cv=5,  scoring='neg_mean_squared_error'))



In [ ]:
cv_5_results.mean()

In [ ]:
poly4 = PolynomialFeatures(4)
X_degree4 = poly4.fit_transform(X)
reg_poly4 = LinearRegression().fit(X_degree4, y)

from sklearn.model_selection import cross_val_score

cv_5_results2  = np.mean(cross_val_score(reg_poly4, X, y, cv=5,  scoring='neg_mean_squared_error'))
cv_5_results2.mean()

In [ ]:
poly6 = PolynomialFeatures(6)
X_degree6 = poly6.fit_transform(X)
reg_poly6 = LinearRegression().fit(X_degree6, y)

from sklearn.model_selection import cross_val_score

cv_5_results3  = np.mean(cross_val_score(reg_poly6, X, y, cv=5,  scoring='neg_mean_squared_error'))
cv_5_results3.mean()

#### Are certain areas more expensive ?

In [ ]:
df_question1['price_per_sqft'] =df_question1["price"]/df_question1["sqft_above"]
 # arrange price/sqft, remove outliers and form bins of size 100
bins= np.arange(np.quantile(df_question1.price_per_sqft, .015) , np.quantile(df_question1.price_per_sqft, 0.9985), step=100)
df_question1['price_per_sqft_range'] = pd.cut(df_question1['price_per_sqft'],bins) #Segment and sort price/sqft into bins 
df_out = df_question1.loc[(df_question1['price_per_sqft'] > 119) & (df_question1['price_per_sqft'] < 500)]
plt.figure(figsize=(20,10))
sns.scatterplot(data=df_question1,x="long", y="lat", hue= 'price_per_sqft_range')
plt.title('Price per Sqft')
plt.show()


#### Looking at price per sqft we can see The central regios is the most expensive , followed by the north and north west .South is the Cheapest

#### Q1:Do houses tend to have more sqft of living in certain parts ?

In [ ]:
df_out2 = df_question1.loc[(df_question1['sqft_above'] > 180) & (df_question1['sqft_above'] < 3240)]
# sns.scatterplot(data=df_question1,x="long", y="lat", hue= 'sqft_living_range')
plt.figure(figsize=(20,10))
plt.scatter(df_out2['long'], df_out2['lat'], c=df_out2['sqft_above'],s=1)
plt.colorbar().set_label('Sqft', fontsize=14)
plt.title('Sqft')
plt.show()


#### We can see houses tend to be bigger in the northwest and smaller in the east .  